In [ ]:
#No confidential information is displayed in this program.
#All data are fake and generated randomly for program testing purposes

In [23]:
# Automation that will run all code once a day at 3:00 am (outside of business hours) & update dataset



In [231]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [232]:
pip install  mpl_toolkits

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement mpl_toolkits (from versions: none)
ERROR: No matching distribution found for mpl_toolkits


In [259]:
pip install gmplot

  Created wheel for gmplot: filename=gmplot-1.2.0-py3-none-any.whl size=143767 sha256=f239377ebcf31538b109bca340b2ca0fcb8f4781c0af1ea12a25bbdf90a08e17
  Stored in directory: c:\users\18195\appdata\local\pip\cache\wheels\a1\bb\61\143790d9935e333669dd6aeef38f7aec9b3a1b7007f47d6fe8
Successfully built gmplot
Note: you may need to restart the kernel to use updated packages.


In [423]:
# Jerome Blanchet - Transport Canada Analysis - Parsing Sitcen Notification Messages

#TODO

#add French - English translator one word at a time.
#variable listing notification receiver lonly
#putting description in current update if original notification
#considering correction notification that actually correct past update or description

# the structure of Sitcen notification may change accross time and notification type. 
# we are assuming as less assumption as possible in the code below.

# list of assumption

###
###
###
###
###
###

from __future__ import print_function
from __future__ import division

from datetime import datetime
import os
import re
import sys
import conda
import time
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame

#import geopandas as gpd
#from shapely.geometry import Point, Polygon

import gmplot

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from mpl_toolkits.axes_grid.anchored_artists import AnchoredDrawingArea
#from mpl_toolkits.basemap import Basemap
from IPython.display import Image

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (GMapPlot, GMapOptions, ColumnDataSource, 
Circle, LogColorMapper, BasicTicker, ColorBar,
DataRange1d, PanTool, WheelZoomTool, BoxSelectTool)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5

from geopy.geocoders import Nominatim
pd.set_option('display.max_columns', None)

#matplotlib.style.use('ggplot')

In [2]:
# python modules version

import numpy
print('numpy: {}'.format(numpy.__version__))
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
import pandas
print('pandas: {}'.format(pandas.__version__))

numpy: 1.18.1
matplotlib: 3.1.3
pandas: 1.0.1


In [415]:
# setting up folders
# the current folder is
os.getcwd()

'C:\\Users\\18195\\Downloads\\CITCEN NOTIFICATIONS'

In [416]:
# No matter the current folder name, let's change it to the real one
os.chdir(r'C:\\Users\\18195\\Downloads\\CITCEN NOTIFICATIONS')

In [422]:
# New current folder is
os.getcwd()

'C:\\Users\\18195\\Downloads\\CITCEN NOTIFICATIONS'

In [427]:
#Building temporary dataset

start_time = time.clock()

i = 1

current = os.getcwd()

dataset = pd.DataFrame(columns=['Notification_ID',
                                'File_Name',
                                'Status',
                                'Geography', 
                                'Date', 
                                'Year', 
                                'Month',
                                'Day',
                                'Type', 
                                'Longitude', 
                                'Latitude', 
                                'Short_Description',
                                'Full_Content',
                                'Long_Description',
                                'Source_of_Reporting',
                                'Current_Transport_Canada_Actions',
                                'Current_Actions_other',
                                'Notification_provided_to',
                                'Next_Steps',
                                'Initial_analysis',
                                'Number_of_Update_Displayed',
                                'Current_Update',
                                'Notification',
                                'Number_of_Word_in_Notification',
                                'Number_of_Paragraphe',
                                'Number_of_Character_in_Notification'])

File_Name = ''
Notification_ID = ''
Geography = '' 
Status = ''
Date = ''
Year = ''
Month = ''
Day = ''
Type = ''
Longitude = ''
Latitude = ''
Short_Description = ''
Full_Content = ''
Long_Description = ''
Source_of_Reporting = ''
Current_Transport_Canada_Actions = ''
Current_Actions_other = ''
Notification_provided_to = ''
Next_Steps = ''
Initial_analysis = ''
Number_of_Update_Displayed = ''
Current_Update = ''
Notification = ''
Number_of_Word_in_Notification = ''
Number_of_Paragraphe = ''
Number_of_Character_in_Notification = ''

for dirname, _, filenames in os.walk(current):
    for filename in filenames:

#----------------------Parsing file title-----------------------------------------------------------------        
                
        #print(os.path.join(dirname, filename))
       
        File_Name = filename
    
        listt = re.split(' - | – ', filename)
        Geography = listt[-2]    
    
        #listt = filename.split(" - ")
        #Geography = listt[-2] 
 
        #search = re.findall(r'[ A-Za-z]+ - \d\d?', filename)
        #search = re.findall(r'[ A-Za-z]+ - \d\d?', filename) #(\w-\w)
        #search = search[0][:-4]
        #Geography = search[1:]
        
        search = re.findall(r'\d\d \w+ \d\d\d\d', filename)
        Date = search[0]
        #Date_Formatted = datetime.strptime(Date,'%Y%m%d')
        Year = search[0][-4:]
        Month = search[0][:-4][2:]
        Day = search[0][:2]
        
        search = re.findall(r'[A-Z]{3}[-/ A-Z]+', filename)
        #search = re.findall(r'[A-Z]{3}[\s]-[\s][A-Z]+', filename)        
        Type = search[0] 
        
        listt = re.split(' - | – ',filename)
        Short_Description = listt[-3]
        
        #try:
            #listt = filename.split(" - ")
            #Short_Description = listt[-3]   
        #except:
            #listt = filename.split(" – ")
            #Short_Description = listt[-3]        
            
        def __status__(string):
        
            try:
                search = re.findall(r'Mise-à-jour [\d]+', string)
                Status = search[0]
            except:
                pass
                try:
                    search = re.findall(r'Mise à jour [\d]+', string)
                    Status = search[0]
                except:
                    pass
                    try:
                        search = re.findall(r'Update [\d]+', string)
                        Status = search[0] 
                    except:      
                        Status = 'Original'  
            
            if Status[:4] == 'Mise':
                Status = 'Update ' + Status[-2:]
            
            return Status
        
        Status = __status__(filename)
        
#----------------------Parsing file internal content-------------------------------------------------------------       
        
        with open(filename, 'r') as myfile: 
            
            data = myfile.read()
            a = [i.strip().split() for i in data.split('\n\n') if len(i.strip())>0]
            
            Full_Content = a
            
        myfile.close()
            
        
        Long_Description = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0][:11] == 'Description']
        
        Source_of_Reporting = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0][:6] == 'Source']
        
        Current_Transport_Canada_Actions = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0] == 'Current' and Full_Content[y][1] == 'Transport']
        
        Current_Actions_other = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0] == 'Current' and Full_Content[y][1] == 'Actions']

        Notification_provided_to = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0] == 'Notification' and Full_Content[y][1] == 'provided']
        
        Next_Steps = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0] == 'Next' and Full_Content[y][1] == 'Steps:']
        
        Initial_analysis = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0] == 'Initial' and Full_Content[y][1] == 'analysis/impact']
        
        
        Number_of_Update_Displayed = len([Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0] == 'Update']) 
        
        List_of_Update_Paragraph = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0] == 'Update']
        
        List_of_Update_Number = []
        
        for t in range(Number_of_Update_Displayed):
        
            List_of_Update_Number.append(List_of_Update_Paragraph[t][1][:-1])
            Max_Update_Number = max(List_of_Update_Number)
            
        Current_Update = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0] == 'Update' and Full_Content[y][1] == Max_Update_Number + ':']
        
        
        Notification = [Full_Content[y] for y in range(len(Full_Content))\
                            if Full_Content[y][0] == 'NOTIFICATION' and Full_Content[y][2] != 'provided']
        
#----------------------World Map Features Engineering----------------------------------------------------------         
        
        geolocator = Nominatim()
        coordonates = geolocator.geocode(Geography)
        Latitude, Longitude = coordonates.latitude, coordonates.longitude        
        
#----------------------Text Mining Features Engineering----------------------------------------------------------         
        
        Number_of_Paragraphe = len(Full_Content)
        Number_of_Word_in_Notification = sum([len(listElem) for listElem in Full_Content])
        Number_of_Character_in_Notification = sum([sum([len(word) for word in listelem]) for listelem in Full_Content])
        
#----------------------Wrapping up everything in a dataset-------------------------------------------------------         
        
        dataset = dataset.append({'File_Name': File_Name,
                                  'Notification_ID': i, 
                                  'Geography': Geography, 
                                  'Status': Status,
                                  'Date': Date,
                                  'Year': Year,
                                  'Month': Month,
                                  'Day': Day,
                                  'Type': Type,
                                  'Longitude': Longitude,
                                  'Latitude': Latitude,
                                  'Short_Description': Short_Description,
                                  'Full_Content': Full_Content,
                                  'Long_Description': Long_Description,
                                  'Source_of_Reporting': Source_of_Reporting,
                                  'Current_Transport_Canada_Actions': Current_Transport_Canada_Actions,
                                  'Current_Actions_other': Current_Actions_other,
                                  'Notification_provided_to': Notification_provided_to,
                                  'Next_Steps': Next_Steps,
                                  'Initial_analysis': Initial_analysis,
                                  'Number_of_Update_Displayed': Number_of_Update_Displayed,
                                  'Current_Update': Current_Update,
                                  'Notification': Notification,
                                  'Number_of_Word_in_Notification': Number_of_Word_in_Notification,
                                  'Number_of_Paragraphe': Number_of_Paragraphe,
                                  'Number_of_Character_in_Notification': Number_of_Character_in_Notification}, ignore_index=True)
        
        File_Name = ''
        Notification_ID = ''
        Geography = '' 
        Status = ''
        Date = ''
        Year = ''
        Month = ''
        Day = ''
        Type = ''
        Longitude = ''
        Latitude = ''
        Short_Description = ''
        Full_Content = ''
        Long_Description = ''
        Source_of_Reporting = ''
        Current_Transport_Canada_Actions = ''
        Current_Actions_other = ''
        Notification_provided_to = ''
        Next_Steps = ''
        Initial_analysis = ''
        Number_of_Update_Displayed = ''
        Current_Update = ''
        Notification = ''
        Number_of_Word_in_Notification = ''
        Number_of_Paragraphe = ''
        Number_of_Character_in_Notification = ''
        
        i += 1
        
end_time = time.clock()

print("")
print("Total Estimation Running Time:")
print(end_time - start_time, "Seconds")

C:\Users\18195\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\18195\Anaconda3\lib\site-packages\ipykernel_launcher.py:185: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.



Total Estimation Running Time:
3.511145399999805 Seconds


C:\Users\18195\Anaconda3\lib\site-packages\ipykernel_launcher.py:253: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [425]:
#Displaying Temporary dataset
dataset.head(10) 

,Notification_ID,File_Name,Status,Geography,Date,Year,Month,Day,Type,Longitude,Latitude,Short_Description,Full_Content,Long_Description,Source_of_Reporting,Current_Transport_Canada_Actions,Current_Actions_other,Notification_provided_to,Next_Steps,Initial_analysis,Number_of_Update_Displayed,Current_Update,Notification,Number_of_Word_in_Notification,Number_of_Paragraphe,Number_of_Character_in_Notification
0,1,(ONT - TDG - UUUU) Correction - Mise-à-jour 44...,Update 44,Emo ON,18 February 2020,2020,February,18,ONT - TDG - UUUU,-93.835698,48.632292,Derailment with Crude Oil,"[[fdaseddf,, tuyjj], [mhjtmhyt, bfdbf], [NOTIF...","[[Description, of, threat/event:, ddfdfsddhr, ...","[[Source, of, reporting:, bbbbbbbbbbbbbb]]","[[Current, Transport, Canada, Actions:, hhhhhh...","[[Current, Actions, (other):, yyyyyyyyyyyyyyyy...","[[Notification, provided, to:, kkkkkkkkkkkkkkk...","[[Next, Steps:, ooooooooooooooooooooooo]]","[[Initial, analysis/impact, assessment, on, Ca...",2,"[[Update, 02:, sdfgsdfsdfsdfgsdfg]]","[[NOTIFICATION, -, TREAT/EVENT:, (ONT, -, TDG)...",83,14,675
1,2,(ONT - TDG - UUUU) Correction - Mise-à-jour 44...,Update 44,Montreal Quebec,18 February 2026,2026,February,18,ONT - TDG - UUUU,-73.610364,45.497216,Derailment with Crude Oil,"[[fdaseddf,, tuyjj], [mhjtmhyt, bfdbf], [NOTIF...","[[Description, of, threat/event:, ddfdfsddhr, ...",[],[],[],[],[],[],0,[],"[[NOTIFICATION, -, TREAT/EVENT:, (ONT, -, TDG)...",49,6,332
2,3,(ONT - TDG) Correction - Update 12 - Immediate...,Update 12,Calgary Alberta,18 February 2021,2021,February,18,ONT - TDG,-114.062589,51.053423,Derailment with Crude Oil,"[[fdaseddf,, tuyjj], [mhjtmhyt, bfdbf], [NOTIF...","[[Description, of, threat/event:, ddfdfsddhr, ...",[],[],[],[],[],[],0,[],"[[NOTIFICATION, -, TREAT/EVENT:, ASDFASDFASDF,...",34,6,272
3,4,(ONT - TDG) Correction - Update 12 - Immediate...,Update 12,Victoria BC,18 February 2025,2025,February,18,ONT - TDG,-123.364953,48.428318,Derailment with Crude Oil,"[[fdaseddf,, tuyjj], [mhjtmhyt, bfdbf], [NOTIF...","[[Description, of, threat/event:, ddfdfsddhr, ...",[],[],[],[],[],[],0,[],"[[NOTIFICATION, -, TREAT/EVENT:, ASDFASDFASDF,...",34,6,272
4,5,(ONT - TDGY) Correction - ggg - Immediate Noti...,Original,Faro Yukon,18 February 2024,2024,February,18,ONT - TDGY,-133.355315,62.229541,Derailment with Crude Oil,"[[fdaseddf,, tuyjj], [mhjtmhyt, bfdbf], [NOTIF...","[[Description, of, threat/event:, ddfdfsddhr, ...",[],[],[],[],[],[],2,"[[Update, 02:, sdfgsdfsdfsdfgsdtyutyif]]","[[NOTIFICATION, -, THREAT/EVENT:, ASDFASDFASDF...",40,8,325
5,6,(ONT - TDGY) Correction - ggg - Immediate Noti...,Original,Toronto ON,18 February 2022,2022,February,18,ONT - TDGY,-79.383935,43.653482,Derailment with Crude Oil,"[[fdaseddf,, tuyjj], [mhjtmhyt, bfdbf], [NOTIF...","[[Description, of, threat/event:, ddfdfsddhr, ...",[],[],[],[],[],[],0,[],"[[NOTIFICATION, -, TREAT/EVENT:, ASDFASDFASDF,...",34,6,272


In [405]:
# Displaying World Map with Sitcen notification geography as an .html file

#centering the map display in Canada with proper zoom
mymap = gmplot.GoogleMapPlotter(57.802818, -103.070289, 4) 

mymap.plot(dataset['Latitude'], dataset['Longitude'], 'red', edge_width=2.5)

#filling trajectories
#mymap.polygon(dataset['Latitude'], dataset['Longitude'], 
                   #color = 'cornflowerblue')

sizee = dataset['Notification_ID'].tolist()

mymap.scatter(dataset['Latitude'], dataset['Longitude'], '# FF0000', size = sizee, marker = True) 

mymap.draw("Sitcen_Notification_Mapping.html") 

os.system("Sitcen_Notification_Mapping.html");

In [ ]:
# Extracting new Sitcen notification from Outlook & storing it as individual txt file in a folder



In [ ]:
# deleting new Sitcen notification from Outlook

